<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg2/C_mixup_lbl_smoothing_cutout_tta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>